<a href="https://colab.research.google.com/github/naokityokoyama/Projetos_Diversos/blob/master/Mini_ETL_Tratamento_Exportacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import csv
import os
import pandas as pd
from dateutil import parser
from datetime import datetime
path = '/tmp/DET/'

#funcao importa 
def importa(link):
  arquivo = open(link)
  linhas = csv.reader(arquivo)
  lista_csv = []
  for linha in linhas:
    lista_csv.append(linha)
  return lista_csv   

#funcao regex
def rege(importado):
  resultado1 = []
  for i in range(1, len(importado)):
    resultado2 = []
    resultado1.append(re.sub(r'$', '.000000Z', importado[i][0]))
    for j in range(len(resultado1)):
      resultado2.append(re.sub(r' ', 'T', resultado1[j]))
  return resultado2

while True:
 for file in os.listdir(path):
   if file.startswith("DET_Export_LOL-SLA_Traceroute_Active"):
    csvi = (os.path.join(path, file))
  
    importado = importa(csvi)
    data_tratada = rege(importado)
  
    #replace
    for i,j in zip(range(1,len(importado)), range(len(data_tratada))):
      importado[i][0]= importado[i][0].replace(importado[i][0], data_tratada[j])
  
    #join 
    final = []
    for i in importado:
      final.append(','.join(i))
    final
    #head
    st1 = '#group false,true,true,true,true,true,true,false,false,false,false,true,false,false,false,false,false,true,true,false'
    st2 = '#datatype time,tag,tag,tag,tag,measurement,tag,long,long,long,long,tag,long,long,long,long,long,tag,tag,string'
    st3 = "Time_Stamp,Test_Instance_ID,Verifier,SLA_ID,SLA_Name,Service_Instance_Name,Interface_Category,Result,Hop_Count_to_Destination,Number_of_ICMP_Replies_From_Destination,Network_Topology_Change,Destination_IP_Address,Minimum_Round-Trip_Latency_to_Destination,Maximum_Round-Trip_Latency_to_Destination,Median_Round-Trip_Latency_to_Destination,Average_Round-Trip_Latency_to_Destination,Standard_Deviation,Host_Tested,Source_Interface_Address,Route_Hash"

    final.insert(0, st1)
    final.insert(0, st2)
    final[2]=st3

    np.savetxt("final.csv", final, fmt='%s')

    influx = 'influx write -b LoL -f final.csv'
    os.system(influx)
    copia = 'cp '+ csvi + ' /tmp/DET/processado/'
    apaga = 'rm '+ csvi
    os.system(copia)
    os.system(apaga)
